### Methods for Data Augmentation

In [93]:
import cv2
import numpy as np
import random
import os
# rotation random angel
def rotation(image_path,image_number):
    img = cv2.imread(image_path)
    for i in range(image_number):
        image_center = tuple(np.array(img.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(image_center, random.randint(-40, 40), 1.0)
        rotated_image = cv2.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv2.INTER_LINEAR)
        output_path = f"{image_path[:-4]}R{i+1}.png"
        cv2.imwrite(output_path, rotated_image)
        
# add Gaussian noise
def add_gaussian_noise(image_path, image_number):
    img = cv2.imread(image_path)
    for i in range(image_number):
        noise = np.random.normal(0, random.randint(1, 5)*5, img.shape).astype(np.int16)
        noisy_image = np.clip(np.round(img + noise), 0, 255).astype(np.uint8)
        output_path = f"{image_path[:-4]}G{i+1}.png"
        cv2.imwrite(output_path, noisy_image)
# Gaussian blur
def gaussian_blur_image(image_path,image_number):
    image = cv2.imread(image_path)
    for i in range(image_number):
        s = (i)
        ksize = (2 * s + 1, 2 * s + 1)
        blurred_image = cv2.GaussianBlur(image, ksize, sigmaX=5)
        output_path = f"{image_path[:-4]}B{i+1}.png"
        cv2.imwrite(output_path, blurred_image)
# brightness
def brighten(image_path,image_number):
    image = cv2.imread(image_path)
    image = image.astype(np.float32)
    for i in range(image_number):
        g = random.randint(2,3)*0.5
        brightened_image = image * g
        brightened_image = np.clip(brightened_image, 0, 255)
        brightened_image = brightened_image.astype(np.uint8)
        output_path = f"{image_path[:-4]}Bt{i+1}.png"
        cv2.imwrite(output_path, brightened_image)
# crop and occlude
def crop_and_occlude(image_path,image_number):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    for i in range(image_number):
        cut_rate = random.randint(10,40)/100
        start_x = int(width * cut_rate)
        end_x = int(width * 1-cut_rate)
        start_y = int(height * cut_rate)
        end_y = int(height * 1-cut_rate)
        cropped_image = image[start_y:end_y, start_x:end_x]
        output_path = f"{image_path[:-4]}Co{i+1}.png"
        cv2.imwrite(output_path, cropped_image)


# rotation("dataset\\AWEDataset\\awe\\001\\01.png",5)
# add_gaussian_noise("dataset\\AWEDataset\\awe\\001\\01.png",5)
# gaussian_blur_image("dataset\\AWEDataset\\awe\\001\\01.png",5)
# brighten("dataset\\AWEDataset\\awe\\001\\01.png",5)
# crop_and_occlude("dataset\\AWEDataset\\awe\\001\\01.png",5)

### Scripts of Data Augmentation

In [96]:
import glob

PATH = "dataset\Customized dataset"
dir_list = []
for root, dirs, files in os.walk(PATH):
    for d in dirs:
        dir_list.append(os.path.join(root,d))


for i in range(len(dir_list)):
    folder_path = dir_list[i]
    file_list=[]
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_list.append(os.path.join(root,file))
    for i in range(len(file_list)-1):
        add_gaussian_noise(file_list[i],6)
        rotation(file_list[i],6)      
        gaussian_blur_image(file_list[i],6)
        brighten(file_list[i],6)
        crop_and_occlude(file_list[i],6)


### Mthods and Scripts for Edge Detection(not used)

In [17]:
def edge_detection(image_path):
    # load image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    height, width = gray.shape[:2]
    left = int(width / 10)
    top = int(height / 10)
    right = width - int(width / 10)
    bottom = height - int(height / 10)
    cropped_image = gray[top:bottom, left:right]
    cropped_image = cv2.resize(cropped_image, (64, 64))
    # edges = cv2.Canny(cropped_image, 50, 150)
    cv2.imwrite(image_path, cropped_image)

for i in range(len(PATH)):
    folder_path = PATH[i]
    file_pattern = os.path.join(folder_path, '*.png')
    for file_path in glob.glob(file_pattern):
        edge_detection(file_path)

### Separate images that do not contain rotations(not used)

In [ ]:
import os
PATH = 'dataset/Customized dataset/validate'
def delete_files_with_r(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if 'R' in file or 'r' in file:
                file_path = os.path.join(root, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f'已删除: {file_path}')
delete_files_with_r(PATH)


In [ ]:
from ear_CNN import EarCNN
from utils import read_split_data
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torchvision import transforms
from my_dataset import MyDataSet
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def main():
    model = EarCNN()
    m_state_dict = torch.load('EAR_CNN.pth')
    model.load_state_dict(m_state_dict)
    model.eval()
    data_transforms = transforms.Compose([
        transforms.Lambda(lambda image: image.convert('RGB')),
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])
    img1 = Image.open('ear_image.png')
    img1 = data_transforms(img1)
    img1 = img1.unsqueeze(0)
    img2 = Image.open('dataset\\Customized dataset\\train\\001\\01B3.png')
    img2 = data_transforms(img2)
    img2 = img2.unsqueeze(0)
    
    with torch.no_grad():
        output1 = model(img1)
        output2 = model(img2)
    print(output1)
    print(output2)


if __name__ == "__main__":
    main()



### Save HRIR data to .json

In [ ]:

from pysofaconventions import SOFAFile
import os
import json


def list_files_in_folder(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_path = file_path.replace('\\', '/')
            file_paths.append(file_path)
    return file_paths


folder_path = 'dataset/cipic-hrtf-database-master/cipic-hrtf-database-master/sofa_files'
file_paths_list = list_files_in_folder(folder_path)


def get_HRIR(path):
    sofa = SOFAFile(path, 'r')
    hrir = sofa.getDataIR()
    return hrir

hrir_dict = {}
for file_path in file_paths_list:
    file_name = os.path.basename(file_path)
    hrir_data = get_HRIR(file_path)
    hrir_dict[file_name] = hrir_data.data.tolist()

# save
output_file = 'hrir_data.json'
with open(output_file, 'w') as json_file:
    json.dump(hrir_dict, json_file)

print(f"HRIR saved to .JSON: {output_file}")
